In [1]:
import pandas as pd
import re
from termcolor import colored

In [ ]:
# Loading Raw Data to Dataframe

In [2]:
raw_Tweets=pd.read_csv('data/rawdata/Tweets.csv')

In [3]:
raw_Tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58045 entries, 0 to 58044
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TweetsID       58045 non-null  int64 
 1   TweetsText     58045 non-null  object
 2   User_Location  45205 non-null  object
 3   Date           58045 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.8+ MB


In [25]:
raw_Tweets.head(2)

,TweetsID,TweetsText,User_Location,Date
0,1216910000000000000,We’re doing everything we can to get our stude...,NaN,24/10/2020 03:29
1,40367314,https://t.co/jbGMWju8fn \n\n Thanks @RobinRobe...,Danny St.,24/10/2020 03:01


  # Data Statistics:    Gaining insight on the  Datasets

In [ ]:
# Checking for NULL VALUES

In [5]:
raw_Tweets.isnull().any()

TweetsID         False
TweetsText       False
User_Location     True
Date             False
dtype: bool

In [6]:
print(colored("\nTest shows that there some null values in User_Location column. \nHence the NULL entries were replaced with word 'LOCATION'\n","red"))


Test shows that there some null values in User_Location column. 
Hence the NULL entries were replaced with word 'LOCATION'



In [ ]:
#raw_Tweets.loc[raw_Tweets['User_Location'].isnull()] #Showing the NULL values

# Processing the raw tweets using function  get_tweet_content()

In [11]:
Tweets=get_tweet_content()

In [12]:
Tweets[0] #Sample of Clean Tweet

{'TweetsID': '1216910000000000000',
 'TweetsText': 'were doing everything we can to get our students back in school safely and quickly post hurricanelaura amp hurricanedelta and are thankful to those whove helped already but we need somebody who can get us tents or something asap catholictwitter ',
 'User_Location': 'LOCATION',
 'Date': '24/10/2020 03:29'}

In [14]:
Tweets_dataFrame=pd.DataFrame(Tweets)

In [15]:
processing_data=Tweets_dataFrame.copy()

In [16]:
processing_data.info() # 36, 727 Retweets and mull values were removed.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21318 entries, 0 to 21317
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TweetsID       21318 non-null  object
 1   TweetsText     21318 non-null  object
 2   User_Location  21318 non-null  object
 3   Date           21318 non-null  object
dtypes: object(4)
memory usage: 666.3+ KB


# Removing Duplicates

In [17]:
processing_data=processing_data.drop_duplicates(subset=['TweetsText'],keep='first') #Removing by TweetsText Column

In [18]:
processing_data=processing_data.drop_duplicates(subset=['TweetsID'],keep='first') #Removing by TweetsText Column

# Performing check to show all duplicate tweets was removed

In [20]:
processing_data[processing_data.duplicated(['TweetsText'], keep='first')]

,TweetsID,TweetsText,User_Location,Date


In [21]:
processing_data[processing_data.duplicated(['TweetsID'], keep='first')]

,TweetsID,TweetsText,User_Location,Date


In [22]:
processing_data.tail() # User_Location for serial no 14790 now reads 'LOCATION'

,TweetsID,TweetsText,User_Location,Date
14787,1327453151914504195,hey user_mention youre missing an excellent pr...,"Starkville, MS",2020-11-14 03:27:48
14788,1327470015055077378,power restored since halloween after zeta stru...,"Biloxi, MS",2020-11-14 04:34:49
14790,1329106772787744770,the adorablest sibling bond user_mention mohsi...,LOCATION,2020-11-18 16:58:42
14791,1329107820969680898,a 10000 grant from the regions foundation will...,"New Orleans, LA",2020-11-18 17:02:52
14792,1329112952381734912,tesla rivian and several ev manufacturers join...,"San Diego, CA",2020-11-18 17:23:16


In [23]:
processing_data.info() #Distincts Tweets is 9,130. 9,255 duplicates have been removed

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8766 entries, 0 to 14792
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TweetsID       8766 non-null   object
 1   TweetsText     8766 non-null   object
 2   User_Location  8766 non-null   object
 3   Date           8766 non-null   object
dtypes: object(4)
memory usage: 342.4+ KB


In [ ]:
# Saving cleaned Tweets to CSV file 8766 distinct tweets

In [24]:
processing_data.to_csv('data/proData/processdata.csv', index = False)

# Corresponding functions used in above experiments

In [8]:
def process_tweet(tweets):
    
    #for twit in tweets:      
    # Convert to lower case
    tweet= tweets.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'user_mention', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', 'RT', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = re.sub(r'[^\w\s]','',tweet)
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    #tweet=[char for char in tweet if char not in string.punctuation]
    words = ''.join(tweet)
    tweet = words
    return tweet

In [9]:
def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet

In [10]:
import csv
def get_tweet_content():
    #import csv
    from datetime import datetime
    lex1=[]
    
    reader = csv.reader(open('data/rawdata/Tweets.csv', 'r',encoding="utf8"))
    csvreader = csv.reader(reader)
    next(reader)
    for line in reader:
        lex={}
        TweetsID, TweetsText,User_Location,Date = line
        lex['TweetsID']=str(TweetsID)
        lex['TweetsText']=TweetsText
        lex['User_Location']=User_Location
        lex['Date']=Date
    # ignore retweets 
        if len(lex['User_Location']) < 1:        
            lex['User_Location']='LOCATION'
        if lex['TweetsText'].startswith('RT') or len(lex['TweetsText']) < 1:
                pass 
        else:
            lex['TweetsText']=process_tweet(TweetsText)      
            lex1.append(lex)
    
    return lex1